<a href="https://colab.research.google.com/github/A-Wpro/ML_Predict_Movie_revenue/blob/main/movieMLpredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data set
https://www.kaggle.com/tmdb/tmdb-movie-metadata

WIP

TODO LIST :
-PCA ?
-scoring and co (2nd try = 0.7296209632715457 score, that okat need to improve)
-find best model (not knn)

In [18]:
from sklearn import datasets
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder# creating instance of one-hot-encoder
from sklearn.metrics import mean_squared_error
from datetime import datetime


# Movie feat :  adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count

In [19]:
movies_metadata=pd.read_csv('/content/drive/My Drive/movies_metadata.csv', sep=',')


movies_metadataWithoutUselessFeat = movies_metadata
movies_metadataWithoutUselessFeat = movies_metadataWithoutUselessFeat.drop(movies_metadataWithoutUselessFeat.columns[[1,3, 4,5,6,8,9,11,12,13,17,18, 19,20,21]], axis=1)


movies_metadataWithoutNaN = movies_metadataWithoutUselessFeat.dropna()
print("\nShape de la frame :\n", movies_metadataWithoutNaN.shape) # on  perd 1500 lignes au lieu de 40k en enlevant 3 feats

#CLEANING DATA : TIMESTAMP TO UNIX SECONDS


tab = list()
for time in movies_metadataWithoutNaN.release_date:
    time = time # + "00:00:00.000000"
    time = datetime.strptime(time, "%Y-%m-%d")
    time = datetime.timestamp(time)
    tab.append(time)
movies_metadataWithoutNaN.release_date = tab
movies_metadataWithoutNaN.release_date =movies_metadataWithoutNaN.release_date.astype(int)




/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)



Shape de la frame :
 (45119, 9)


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [20]:
#les dummies; on regarde deja commetn ca se passe
#dummies
#movies_metadataWithoutNaN=pd.get_dummies(movies_metadataWithoutNaN, columns=['adult','original_language'])#,''])

from sklearn.preprocessing import LabelEncoder
LabelEnc = LabelEncoder()  
movies_metadataWithoutNaN['adult'] = LabelEnc.fit_transform(movies_metadataWithoutNaN["adult"])
movies_metadataWithoutNaN['original_language'] = LabelEnc.fit_transform(movies_metadataWithoutNaN["original_language"])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
X = movies_metadataWithoutNaN.drop(['revenue'], axis = 1) #5= revenue; if revenue = 0 then no money was win
y = movies_metadataWithoutNaN.revenue              
print('y :',y.shape)
print('x :',X.shape)
print(X.columns)
print(y)

y : (45119,)
x : (45119, 8)
Index(['adult', 'budget', 'original_language', 'popularity', 'release_date',
       'runtime', 'vote_average', 'vote_count'],
      dtype='object')
0        373554033.0
1        262797249.0
2                0.0
3         81452156.0
4         76578911.0
            ...     
45460            0.0
45462            0.0
45463            0.0
45464            0.0
45465            0.0
Name: revenue, Length: 45119, dtype: float64


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3, random_state=42)

In [85]:
print((X_train.iloc[0]))
#adult', 'budget', 'original_language', 'popularity', 'release_date','runtime', 'vote_average', 'vote_count'
# 0 0 17 3.5 1009843200 89.0 8.3 11

adult                        0
budget                       0
original_language           17
popularity            0.005351
release_date         550454400
runtime                     95
vote_average               4.5
vote_count                   1
Name: 4008, dtype: object


In [90]:


RegLin = LinearRegression()
RegLin.fit(X_train, y_train)
y_pred = RegLin.predict(X_test)

print("Score : ", RegLin.score(X_test, y_test))

print("MSE" ,  mean_squared_error(y_test, y_pred))
#MSE = 0.00012 => probably overfited :( I will try to clean data

# TESTING random predict
test = [0,0,17,3.5,1009843200,89.0,8.3,11]
y_pred_test = RegLin.predict(X_train.iloc[0].format(Series))

Score :  0.7296209632715457
MSE 1139717612362195.5


AttributeError: ignored